In [1]:
import pandas as pd
import re

In [2]:
dataframe = pd.read_excel('Amostras.xlsx')
dataframe_teste = pd.read_excel('Dados_Amostra_Teste.xlsx')

In [3]:
def remove_special_characters(text):
        texto_sem_caracteres = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        texto_formatado = re.sub(r'[0-9]', '', texto_sem_caracteres)
        return texto_formatado

In [4]:
def text_formatt(text):

    new_texto = text.split()
    pattern = " ".join(new_texto)
    return pattern

In [6]:
dicionario_positivos = [
    "ASPIROU","ASPIREI","ASPIRO","ASPIRADO","ASPIRAR",
    "ASPIRACAO","SECRECAO","TRACHCARE","VAS","TRACH","CARE",
    "VIAS SUPERIORES","AEREAS","SECRECAO","ASP","SISTEMA FECHADO",
    "VENTILACAO MECANICA","TOT","TQT", "SISTEMA ASPIRACAO","SISTEMA","FECHADO","ENTUBADO","INTUBADO"
]

dicionario_negativos = [
    "EUPNEICO", "AR AMBIENTE","EUPNEICA",
    "EUPENEICA", "EUPENEICO", "EUPONEICO","BRONCOSAPIRACAO"
]

In [7]:
def extrair_contexto_por_palavras(texto, pre=4, pos=3):
    if not isinstance(texto, str):
        return None
    # separa em palavras
    palavras = texto.split()
    
    resultados_positivos = []
    resultados_negativos = []

    # percorre palavras
    i = 0
    while i < len(palavras):
        # verifica se o termo positivo bate a partir da posição i
        for p in palavras:
            if p in dicionario_negativos:
                resultados_negativos.append(p)
        for p in dicionario_positivos:
            termo_palavras = p.split()
            tamanho = len(termo_palavras)
            if palavras[i:i+tamanho] == termo_palavras:
                # pega 'pre' palavras antes e 'pos' palavras depois
                inicio = max(0, i - pre)
                fim = min(len(palavras), i + tamanho + pos)
                contexto = " ".join(palavras[inicio:fim])
                resultados_positivos.append(contexto)
                i += tamanho - 1  # pula as palavras do termo encontrado
                break
        i += 1
    if resultados_positivos:
     return " ".join(resultados_positivos)

    elif resultados_positivos:
        return " ".join(resultados_negativos)
    else:
        return " ".join(palavras)

In [17]:
dataframe['DS_EVOLUCAO'] = dataframe['DS_EVOLUCAO'].apply(remove_special_characters)
dataframe['DS_EVOLUCAO'] = dataframe['DS_EVOLUCAO'].apply(text_formatt)
dataframe.loc[dataframe['VALIDACAO'] == "SIM",'VALIDACAO'] = "S"
dataframe.loc[dataframe['VALIDACAO'] == "NÃO",'VALIDACAO'] = "N"

dataframe_teste['DS_EVOLUCAO'] = dataframe_teste['DS_EVOLUCAO'].apply(remove_special_characters)
dataframe_teste['DS_EVOLUCAO'] = dataframe_teste['DS_EVOLUCAO'].apply(text_formatt)
dataframe_teste['CONTEXTO'] = dataframe_teste['DS_EVOLUCAO'].apply(extrair_contexto_por_palavras)
dataframe_teste.head()

,DS_EVOLUCAO,VALIDACAO,CONTEXTO
0,ADMISSA DE ENFERMAGEM UTI CLINICA PROCEDENTE D...,S,PARA PREVENCAO DE QUEDAS TQT EM USO DE RESPIRA...
1,MT ADMITIDA NA UTI CLINICA I POR DESCONFORTO R...,N,MT ADMITIDA NA UTI CLINICA I POR DESCONFORTO R...
2,FISIOTERAPIANOTURNO EM IH NA UTI POR DESCONFOR...,S,PULMONAR IRPA HIPOXEMICA IOT TQT VM RNC AE COR...
3,EVOLUCAO DE ENFERMAGEM UTI CLINICA PLANTAO NOT...,S,PARA PREVENCAO DE QUEDAS TQT EM USO DA RESPIRA...
4,ANOTACAO DE ENFERMAGEM SNPACIENTE NA UTI POR D...,N,DE PULSOPACIENTE EVOLUI EM TQT SOB VMDIETA POR


In [9]:
dataframe['VALIDACAO'].value_counts()

VALIDACAO
N    1066
S     517
Name: count, dtype: int64

In [10]:
dataframe_teste['VALIDACAO'].value_counts()

VALIDACAO
N    796
S    230
Name: count, dtype: int64

In [11]:
termos = ['TOT','TQT','ENTUBADO','INTUBADO']
pattern = "|".join(termos)

dataframe_conv = dataframe.copy()
dataframe_test_conv = dataframe_teste.copy()

dataframe_conv.loc[(dataframe_conv['VALIDACAO'] == "N") & (dataframe_conv['DS_EVOLUCAO'].str.contains(pattern)),'VALIDACAO'] = 'S'
dataframe_test_conv.loc[(dataframe_test_conv['VALIDACAO'] == "N") & (dataframe_test_conv['DS_EVOLUCAO'].str.contains(pattern)),'VALIDACAO'] = 'S'

In [12]:
dataframe_test_conv['VALIDACAO'].value_counts()

VALIDACAO
N    670
S    356
Name: count, dtype: int64

In [ ]:
dataframe_conv.head()

In [ ]:
dataframe_conv.to_excel('Amostra_split.xlsx',index=False)
dataframe_test_conv.to_excel('Amostra_split_teste.xlsx',index=False)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer

corpus = dataframe_test_conv['CONTEXTO'].to_list()


In [21]:
v = TfidfVectorizer()

v.fit(corpus)
transformed_output = v.fit_transform(corpus)
print(v.vocabulary_)

df_vector = pd.DataFrame(
    {'CHAVE': v.vocabulary_.keys(),
     'QTD': v.vocabulary_.values()}

)

{'para': 3518, 'prevencao': 3882, 'de': 1233, 'quedas': 4059, 'tqt': 4849, 'em': 1587, 'uso': 4983, 'respiratorio': 4255, 'confortavel': 983, 'sendo': 4476, 'realizado': 4110, 'aspiracao': 384, 'pelo': 3572, 'trach': 4853, 'care': 729, 'conforme': 982, 'necessidade': 3204, 'filtros': 1937, 'bacteriologicos': 499, 'sistema': 4533, 'fechadao': 1915, 'fixacao': 1987, 'com': 932, 'traqueofixtorax': 4875, 'simetrico': 4520, 'sc': 4433, 'pedido': 3567, 'home': 2325, 'suspenso': 4643, 'por': 3788, 'pico': 3693, 'toracico': 4831, 'troca': 4902, 'su': 4598, 'urnct': 4971, 'segue': 4462, 'mt': 3146, 'admitida': 127, 'na': 3181, 'uti': 4987, 'clinica': 858, 'desconforto': 1310, 'ih': 2389, 'pneumoniacomorbidade': 3753, 'fibrose': 1929, 'pulmonar': 3971, 'hipotireoidismo': 2303, 'rim': 4325, 'unicoalergia': 4940, 'negaprecaucao': 3228, 'contatoas': 1040, 'recebo': 4134, 'paciente': 3467, 'procedente': 3900, 'do': 1448, 'sulconscientecontactuandocooperativoaceitando': 4621, 'dieta': 1377, 'enteral'

In [17]:
df_vector.to_excel('df_vector.xlsx',index=False)

In [22]:
all_features_names = v.get_feature_names_out()

for word in all_features_names:
    index = v.vocabulary_.get(word)
    print(f"{word} {v.idf_[index]}")

aa 3.233916844136142
aaceitacao 7.241250029368613
aacordado 7.241250029368613
aaem 6.548102848808668
aainstalo 7.241250029368613
aamantendo 6.835784921260449
aapossui 7.241250029368613
ab 6.835784921260449
abaixo 6.548102848808668
abd 6.142637740700503
abdmen 6.835784921260449
abdminalobstrucao 7.241250029368613
abdome 2.897444607514929
abdomen 6.835784921260449
abdominal 3.4800499136750505
abdominalnega 7.241250029368613
aberta 5.2953398803133
aberto 5.1011838658723425
abertura 5.988487060873245
abocath 6.548102848808668
abome 6.324959297494458
aborh 5.161808487688777
abril 6.324959297494458
abrilalergia 5.737172632592339
abscesso 4.989958230762118
abumina 7.241250029368613
abundante 7.241250029368613
abvds 5.737172632592339
ac 6.548102848808668
acada 7.241250029368613
acamada 5.536501937130188
acamado 5.449490560140558
aceita 5.8549556682487225
aceitacao 3.3192766930872986
aceitacaoabdome 5.737172632592339
aceitacaoacesso 6.835784921260449
aceitacaoaferidos 7.241250029368613
aceitaca